In [ ]:
import requests
import re
import socket
import nmap
from bs4 import BeautifulSoup

def crawl_website(url, n):
    visited = set()
    to_visit = [url]
    level = 0

    while to_visit and level <= n:
        current_url = to_visit.pop(0)
        visited.add(current_url)
        try:
            response = requests.get(current_url)
            soup = BeautifulSoup(response.text, "html.parser")
            links = [link.get("href") for link in soup.find_all("a")]
            for link in links:
                if link not in visited and link not in to_visit:
                    to_visit.append(link)
        except:
            pass
        level += 1
    return visited


def detect_sql_injection(url):
    payloads = [
        "'",
        "';",
        '"',
        '";',
        "or 1=1--",
        "or 1=1#",
        "or 1=1/*",
        "admin'--",
        "admin'#",
        "admin'/*",
        "admin' or '1'='1",
        "admin' or 1=1--",
        "admin' or 1=1#",
        "admin' or 1=1/*",
    ]
    for payload in payloads:
        injection_url = url + payload
        try:
            response = requests.get(injection_url)
            if "error" in response.text.lower() or "syntax" in response.text.lower():
                return "SQL injection vulnerability detected with payload: " + payload
        except:
            pass
    return "No SQL injection vulnerabilities detected"


def detect_open_ports(url):
    host = url.split("//")[1].split("/")[0]
    try:
        ip = socket.gethostbyname(host)
        nm = nmap.PortScanner()
        nm.scan(ip, arguments="-T4 -F")
        open_ports = [
            str(port) + "/" + nm[ip]["tcp"][port]["name"]
            for port in nm[ip].all_tcp()
            if nm[ip]["tcp"][port]["state"] == "open"
        ]
        if open_ports:
            return "Open ports detected on host: " + host + "\n" + "\n".join(open_ports)
        else:
            return "No open ports detected on host: " + host
    except:
        return "Could not detect open ports on host: " + host


def detect_xss(urls):
    for url in urls:
        try:
            response = requests.get(url)
            if "<script>" in response.text.lower():
                print(f"[WARNING] Possible XSS vulnerability in: {url}")
        except:
            pass


def detect_remote_file_inclusion(urls):
    for url in urls:
        try:
            response = requests.get(url)
            if (
                "http://" in response.text.lower()
                or "https://" in response.text.lower()
            ):
                print(f"[WARNING] Possible RFI vulnerability in: {url}")
        except:
            pass


def detect_regular_expression_dos(urls):
    for url in urls:
        try:
            response = requests.get(url)
            if re.search(r"(a+){10,}", response.text):
                print(
                    f"[WARNING] Possible regular expression DoS vulnerability in: {url}"
                )
        except:
            pass


url = input("Enter the URL to crawl: ")
n = int(input("Enter the number of levels to crawl: "))

visited_urls = crawl_website(url, n)
detect_xss(visited_urls)
detect_remote_file_inclusion(visited_urls)
detect_regular_expression_dos(visited_urls)
print(detect_sql_injection(url))
print(detect_open_ports(url))


Enter the URL to crawl: https://bwapp.hakhub.net/login.php
Enter the number of levels to crawl: 2
[WARNING] Possible RFI vulnerability in: https://bwapp.hakhub.net/login.php
No SQL injection vulnerabilities detected
Could not detect open ports on host: bwapp.hakhub.net
